## 自动差分引擎 autograd

In [ ]:
#对于此示例，我们从torchvision加载了经过预训练的 resnet18 模型。 我们创建一个随机数据张量来表示具有 3 个通道的单个图像，高度&宽度为 64，其对应的label初始化为一些随机值。

import torch, torchvision
model = torchvision.models.resnet18(pretrained=True)
data = torch.rand(1, 3, 64, 64)
labels = torch.rand(1, 1000)


#接下来，我们通过模型的每一层运行输入数据以进行预测。 这是正向传播。
prediction = model(data) # forward pass


#我们使用模型的预测和相应的标签来计算误差（loss）。 下一步是通过网络反向传播此误差。 当我们在误差张量上调用.backward()时，开始反向传播。 然后，Autograd 会为每个模型参数计算梯度并将其存储在参数的.grad属性中。
loss = (prediction - labels).sum()
loss.backward() # backward pass


#接下来，我们加载一个优化器，在本例中为 SGD，学习率为 0.01，动量为 0.9。 我们在优化器中注册模型的所有参数。
optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)


#最后，我们调用.step()启动梯度下降。 优化器通过.grad中存储的梯度来调整每个参数。
optim.step() #gradient descent


## Autograd 的微分


In [ ]:
import torch

a = torch.tensor([2., 3.], requires_grad=True) #要求跟踪a,b的操作
b = torch.tensor([6., 4.], requires_grad=True)

Q = 3*a**3 - b**2   #Q为误差

external_grad = torch.tensor([1., 1.])
Q.backward(gradient=external_grad)

# check if collected gradients are correct
print(9*a**2 == a.grad)
print(-2*b == b.grad)


In [ ]:
import torch
from torch.autograd import Variable

#生成一个内容为[2,3]的张量，Varibale 默认是不要求梯度的，如果要求梯度，
#需要加上requires_grad=True来说明
#这里的Variable是为了设置变量，把a0=2,a1=3设置为两个变量
a = Variable(torch.tensor([2.,3.]),requires_grad=True)
b = a+3
c = b*3
out=c.mean() #求均值
out.backward()
print("a=",a)
print("out=",out)
print(a.grad)  #求out对a的偏导


In [ ]:

#生成一个内容为[2,4]的张量，Varibale 默认是不要求梯度的，如果要求梯度，
#需要加上requires_grad=True来说明
a = Variable(torch.Tensor([[2,4]]),requires_grad=True)
b = torch.zeros(1,2)
b[0,0] = a[0,0]**2+a[0,1]
b[0,1] = a[0,1]**3+a[0,0]
out = 2*b

#括号里面的参数要传入和out维度一样的矩阵
#这个矩阵里面的元素会作为最后加权输出的权重系数
out.backward(torch.FloatTensor([[1, 0]]),retain_graph=True)
A_temp = a.grad.clone()
a.grad.zero_()
out.backward(torch.FloatTensor([[0, 1]]))
B_temp = a.grad
print('jacobian matrix is:')
print(torch.cat((A_temp, B_temp)))

# out.backward(torch.FloatTensor([[1,2]]))
# print("a=",a)
# print("out=",out)
# print(a.grad)  #求out对a的偏导


In [ ]:
import torch
from torch.autograd import Variable

#生成一个内容为[2,3]的张量，Varibale 默认是不要求梯度的，如果要求梯度，
#需要加上requires_grad=True来说明
a = Variable(torch.Tensor([[2,3],[1, 2]]),requires_grad=True)
w = Variable(torch.ones(2,2),requires_grad=True)
out = torch.mm(a,w)

#括号里面的参数要传入和out维度一样的矩阵
#这个矩阵里面的元素会作为最后加权输出的权重系数
out.backward(torch.FloatTensor([[1,1],[1,1]]))
print("gradients are:{}".format(w.grad.data))


## 从 DAG 中排除
>torch.autograd跟踪所有将其requires_grad标志设置为True的张量的操作。 对于不需要梯度的张量，将此属性设置为False会将其从梯度计算 DAG 中排除。

>即使只有一个输入张量具有requires_grad=True，操作的输出张量也将需要梯度。

In [ ]:
x = torch.rand(5, 5)
y = torch.rand(5, 5)
z = torch.rand((5, 5), requires_grad=True)

a = x + y
print(f"Does `a` require gradients? : {a.requires_grad}")
b = x + z
print(f"Does `b` require gradients? : {b.requires_grad}")
